In [2]:
# Step 1: importing required libraries
import tensorflow as tf
from transformers import BertTokenizer, TFBertForNextSentencePrediction
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import numpy as np
import pandas as pd

In [19]:
# Step 2: loading Data from a labeled dataset
df = pd.read_csv('/Users/debabratapanda/Desktop/nsplabel.csv')
df = df[df != '#NAME?']
df

,sentence 1,sentence 2,label
0,"If, then, they have no power, why dost thou pr...",Where is it then? It is in that part of thee i...,1
1,But if they determine about nothing- which it ...,"The soul of man does violence to itself, first...",1
2,He who has a vehement desire for posthumous fa...,"Suppose that men kill thee, cut thee in pieces...",1
3,Things are in such a kind of envelopment that ...,Has any obstacle opposed thee in thy efforts t...,1
4,So then that which is a hindrance to the intel...,But he who has failed in any one of these thin...,1
...,...,...,...
502,He did not take the bath at unseasonable hours...,"His dress came from Lorium, his villa on the c...",0
503,"To-day I have got out of all trouble, or rathe...",Remember that this which pulls the strings is ...,1
504,But the motion of virtue is in none of these: ...,Dost thou not see how many qualities thou art ...,1
505,What kind of things those are which appear goo...,For if any man should conceive certain things ...,0


In [20]:
# making lists of inputs and defining datatype 
s1 = df['sentence 1'].astype(str).tolist()
s2 = df['sentence 2'].astype(str).tolist()
lb = df['label'].astype(int).tolist()

In [22]:
# Step 3: tokenization of sentences 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
inp = tokenizer(s1, s2, return_tensors='tf', padding=True, truncation=True)

In [23]:
# Step 4: selecting a pretrained BERT model for the model creation and further learning 
model = TFBertForNextSentencePrediction.from_pretrained('bert-base-uncased')

All PyTorch model weights were used when initializing TFBertForNextSentencePrediction.

All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


In [24]:
# configuring an input element combining all required elements from our tokenized input for training dataset creation   
ip = (inp['input_ids'], inp['token_type_ids'], inp['attention_mask'])

In [25]:
# Step 5: Creating a dataset from the input tensors and labels
dt = tf.data.Dataset.from_tensor_slices((ip, lb)).batch(32)

In [26]:
# Step 6: Model compilation and parameter settings 
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), loss=SparseCategoricalCrossentropy(from_logits=True))

In [28]:
# Step 7: fitting our defined model on the created dataset
model.fit(dt, epochs=7)

Epoch 1/7
16/16 [==============================] - 1178s 73s/step - loss: 0.7131
Epoch 2/7
16/16 [==============================] - 1134s 71s/step - loss: 0.6599
Epoch 3/7
16/16 [==============================] - 1141s 71s/step - loss: 0.6190
Epoch 4/7
16/16 [==============================] - 1138s 71s/step - loss: 0.5645
Epoch 5/7
16/16 [==============================] - 1128s 70s/step - loss: 0.4914
Epoch 6/7
16/16 [==============================] - 1129s 71s/step - loss: 0.3776
Epoch 7/7
16/16 [==============================] - 1131s 70s/step - loss: 0.2869


In [29]:
# Step 8: Evaluating the model by creating a validation set from our loaded data and calculating loss on it 
vi = tokenizer(s1, s2, return_tensors='tf', padding=True, truncation=True)
vl = np.array(lb)
vt = (vi['input_ids'], vi['token_type_ids'], vi['attention_mask'])
vd = tf.data.Dataset.from_tensor_slices((vt, vl)).batch(32)

In [30]:
ev = model.evaluate(vd)
print("Validation Loss:", ev)

16/16 [==============================] - 259s 16s/step - loss: 0.1795
Validation Loss: 0.1795121431350708
